# 해당 코드는 원본 MIMIC-III 데이터에서, label.csv와 data.csv(모델 사전학습 데이터)를 생성하는 코드입니다. 

따라서 원본 MIMIC-III 데이터의  

ADMISSIONS.csv, PATIENTS.csv, LABEVENTS.csv, PROCEDUREEVENTS_MV.csv, PRESCRIPTIONS.csv, D_ICD_DIAGNOSES.csv, DIAGNOSES_ICD.csv  

가 같은 폴더에 있을 때에만 정상적으로 실행 가능합니다.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime as dt
from tqdm import tqdm

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# ADMISSIONS.csv : 환자의 입/퇴원 기록
admissions = pd.read_csv('ADMISSIONS.csv')

# PATIENTS.csv : 환자의 정보
patients = pd.read_csv('PATIENTS.csv')

# LABEVENTS.csv : 채혈 등의 검사로 나타나는 검사결과
labevents = pd.read_csv('LABEVENTS.csv')

# PROCEDUREEVENTS_MV.csv : 환자의 시술 목록
pro = pd.read_csv('PROCEDUREEVENTS_MV.csv')

# PRESCRIPTIONS.csv : 환자의 처방 목록
pre = pd.read_csv('PRESCRIPTIONS.csv')

# D_ICD_DIAGNOSES.csv : 질병의 종류와 ICD9 CODE 등의 정보
diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv')

# DIAGNOSES_ICD.csv : 환자의 질병 정보
diagnoses_icd = pd.read_csv('DIAGNOSES_ICD.csv')



# -----------------------------------------------------------------------------------
# # 확인용. 실행시키지 말 것
# admissions = pd.read_csv('/project/datacamp/team8/data/ADMISSIONS.csv')
# patients = pd.read_csv('/project/datacamp/team8/data/PATIENTS.csv')
# labevents = pd.read_csv('/project/datacamp/team8/data/LABEVENTS.csv')
# pro = pd.read_csv('/project/datacamp/team8/data/PROCEDUREEVENTS_MV.csv')
# pre = pd.read_csv('/project/datacamp/team8/data/PRESCRIPTIONS.csv')

# diagnoses = pd.read_csv('/project/datacamp/team8/data/D_ICD_DIAGNOSES.csv')
# diagnoses_icd = pd.read_csv('/project/datacamp/team8/data/DIAGNOSES_ICD.csv')




/home/t8user1/.conda/envs/example/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 패혈증 필터링

In [2]:
#패혈증(Sepsis, Septicemia)과 관련된 질병을 D_ICD_DIAGNOSES.csv의 목록에서 추출합니다. 
diag_sepsis = diagnoses[(diagnoses['SHORT_TITLE'].str.contains('Sepsis')|(diagnoses['SHORT_TITLE'].str.contains('sepsis'))
                         |(diagnoses['SHORT_TITLE'].str.contains('Septicemia'))|(diagnoses['SHORT_TITLE'].str.contains('septicemia')))]

In [3]:
# DIAGNOSES_ICD.csv에서 diag_sepsis로 인덱싱하여 패혈증 환자를 추려냅니다.
# value_counts를 통해, 패혈증 환자의 수를 알아봅니다.
sepsis_id = diagnoses_icd[diagnoses_icd['ICD9_CODE'].isin(diag_sepsis['ICD9_CODE'])].reset_index()
sepsis_id['ICD9_CODE'].value_counts()

99592    3912
0389     3725
99591    1272
03842     467
03849     395
0380      376
77181     225
0388      206
03843     127
03812     118
0383      110
0382       88
03840      60
03844      27
03841       4
0545        2
0031        1
Name: ICD9_CODE, dtype: int64

- 가장 수가 많고 패혈증임을 확신할 수 있는 99592, 0389, 99591만 사용하기로 결정했습니다.

In [4]:
#[:3]의 Diagnoses_icd 데이터 추출
sepsis = sepsis_id[(sepsis_id['ICD9_CODE'].isin(sepsis_id['ICD9_CODE'].value_counts()[:3].index))].drop(['index','ROW_ID','SEQ_NUM'],axis=1).reset_index(drop=True)

# 환자의 id만 추출하여 환자id에 저장, 총 예측에 사용할 환자 수 확인
환자id = sepsis['SUBJECT_ID'].unique()

# label.csv

- 패혈증 환자의 퇴원일과 사망여부를 label.csv로 저장합니다.

In [5]:
입퇴원 = admissions[admissions['SUBJECT_ID'].isin(환자id)]
label = 입퇴원[['SUBJECT_ID', 'DISCHTIME', 'HOSPITAL_EXPIRE_FLAG']].sort_values('SUBJECT_ID')

#시간 부분 제거, date 부분만 남김
label['DISCHTIME'] = pd.to_datetime(label['DISCHTIME'])
label['DISCHTIME'] = label['DISCHTIME'].dt.date

In [6]:
label.rename(columns={'HOSPITAL_EXPIRE_FLAG': 'LABEL_DEAD1_ALIVE0'}, inplace=True)
label.rename(columns={'DISCHTIME': 'DISCHARGE'}, inplace=True)
label.sort_values(by=['SUBJECT_ID', 'DISCHARGE'], inplace=True)

In [7]:
label.head()

,SUBJECT_ID,DISCHARGE,LABEL_DEAD1_ALIVE0
212,3,2101-10-31,0
229,21,2134-09-24,0
230,21,2135-02-08,1
11,33,2116-12-27,0
19,38,2166-09-04,0


In [8]:
label.to_csv('label.csv')

# data.csv

## 패혈증환자 추출

- labevents에서 패혈증환자 id만을 추출하여 변수 lab에 저장합니다.

In [9]:
lab = labevents[labevents['SUBJECT_ID'].isin(환자id)].reset_index(drop=True)

# value_counts()를 위해 결측치를 문자열 'nan' 으로 설정
lab['FLAG'] = lab['FLAG'].fillna('nan')

- 날짜만을 가지고 분석을 실시할 예정이므로 다른 정보를 제거하는 함수를 작성합니다.

In [10]:
# 시각 데이터에 날짜만 포함되게 전처리하는 함수
def date_only(df,x):
    df[x] = pd.to_datetime(df[x])
    df[x] = df[x].dt.date

## lab

- LABEVENTS.csv에서 SUBJECT_ID/ITEMID/CHARTTIME/FLAG 열을 가져옵니다.
- 이 중 FLAG가 비정상으로 나타나는 값('비정상'으로 나타난 검사결과)만 인덱싱하여 lab2에 저장합니다.

In [11]:
# lab
lab2 = lab.copy()
lab2 = lab2[['SUBJECT_ID','ITEMID','CHARTTIME','FLAG']]
lab2 = lab2[lab2['FLAG']=='abnormal']
date_only(lab2,'CHARTTIME')
lab2['TYPE']='LAB'
lab2

,SUBJECT_ID,ITEMID,CHARTTIME,FLAG,TYPE
4,3,50808,2101-10-12,abnormal,LAB
15,3,50912,2101-10-13,abnormal,LAB
16,3,50931,2101-10-13,abnormal,LAB
21,3,51006,2101-10-13,abnormal,LAB
22,3,51009,2101-10-13,abnormal,LAB
...,...,...,...,...,...
7551815,96443,50862,2109-12-30,abnormal,LAB
7551816,96443,50863,2109-12-30,abnormal,LAB
7551819,96443,50878,2109-12-30,abnormal,LAB
7551821,96443,50885,2109-12-30,abnormal,LAB


- 이후 다른 csv 파일에서의 인덱싱에 활용하기 위해 lab의 SUBJECT_ID 열에서 나타나는 ID 값을 sub_list 변수로 저장합니다.

In [12]:
# lab의 SUBJECT_ID 열을 활용해 데이터 인덱싱
sub_list = lab['SUBJECT_ID'].unique()

- PROCEDUREEVENTS_MV.csv, PRESCRIPTIONS.csv에서도 전처리 과정을 통해 lab2와 같은 열을 가진 데이터프레임 pro2, pre2를 만들어, 3개의 데이터프레임을 병합합니다.
- LABEVENTS.csv와 달리, PRESCRIPTIONS.csv와 PROCEDUREEVENTS_MV.csv에는 CAHRTTIME이 없고, 각각 STARTTIME과 ENDTIME에 이벤트가 일어나고 종료된 시각이 표시되어 있습니다. 이를 하루하루 분리하여 CHARTTIME 열을 갖는 데이터프레임으로 만듭니다.

## pre

- sub_list를 활용해 PRESCRIPTIONS.csv에서 데이터를 인덱싱하여 pre2 데이터프레임을 생성하는 과정입니다.
- PRESCRIPTIONS.csv의 ENDDATE(처방이 끝난 날짜) 열에는 결측치가 다수 존재합니다. 이를 처방이 시작된 날짜(STARTDATE)와 동일한 날짜에 종료된 것으로 해석하여 STARTDATE의 값을 삽입합니다. 
- PRESCRIPTIONS.csv에서는 ITEMID가 존재하지 않으므로 NDC(미 의약품 코드)를 고유 식별자로 활용하고, ITEMID 열에 삽입합니다.

In [13]:
pre2 = pre.copy()


pre2 = pre2[pre2['SUBJECT_ID'].isin(sub_list)]
pre2 = pre2[['SUBJECT_ID','STARTDATE','ENDDATE','NDC']]

date_only(pre2,'STARTDATE')
date_only(pre2,'ENDDATE')

both_null = pre2[(pre2['STARTDATE'].isnull())&(pre2['ENDDATE'].isnull())].index
ndc_null = pre2[(pre2['NDC'].isnull())].index
pre2 = pre2.drop(index=both_null)
pre2 = pre2.drop(index=ndc_null)


# ENDADATE가 null인 값들에 STARTDATE 넣기
end_null = pre2[pre2['ENDDATE'].isnull()]
end_null['ENDDATE'] = end_null['STARTDATE']

start_null = pre2[(pre2['STARTDATE'].isnull())&(pre2['ENDDATE'].notnull())]
start_null['STARTDATE'] = start_null['ENDDATE']

pre2.loc[end_null.index] = end_null
pre2.loc[start_null.index] = start_null



pre2 = pre2.sort_values(['SUBJECT_ID','STARTDATE']).reset_index(drop=True)

/home/t8user1/.conda/envs/example/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/t8user1/.conda/envs/example/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
pre2

,SUBJECT_ID,STARTDATE,ENDDATE,NDC
0,21,2134-09-11,2134-09-11,3.380550e+08
1,21,2134-09-11,2134-09-11,0.000000e+00
2,21,2134-09-11,2134-09-11,6.074928e+06
3,21,2134-09-11,2134-09-12,5.175710e+08
4,21,2134-09-11,2134-09-13,8.084199e+06
...,...,...,...,...
1058004,99991,2185-01-04,2185-01-04,4.560663e+08
1058005,99991,2185-01-04,2185-01-05,4.560663e+08
1058006,99991,2185-01-04,2185-01-05,5.817702e+10
1058007,99991,2185-01-05,2185-01-05,6.348106e+10


In [15]:
# 시작 날짜(start)와 종료 날짜(end)를 입력받아 그 사이에 있는 모든 날짜를 문자열 형태로 리스트에 담아 반환하는 함수
# 예를 들어, start = '2023-08-01'과 end = '2023-08-03'을 입력하면 함수는 ['2023-08-01', '2023-08-02', '2023-08-03']를 반환
def date_range(start, end):
    dates = [(start + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((end-start).days+1)]
    return dates

In [16]:
pre2_list = pre2.values.tolist()
pre2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []

# pre2_list의 각 원소(즉, 각 처방에 대한 정보)를 순회합니다.
# sub, start, end, itemid는 각각 SUBJECT_ID, STARTDATE, ENDDATE, NDC 정보입니다.
for idx, row in enumerate(tqdm(pre2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    # {0: ['2149-11-09'], ...,1327249: ['2181-02-11', '2181-02-12']}
    pre2_dic[idx] = day_list
    
    # pre2 DATE열에 들어갈 lst
    lst_time.extend(pre2_dic[idx][:])

    # pre2 itemid(NDC)열에 들어갈 lst
    for _ in pre2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pre2 subid열에 들어갈 lst
    for _ in pre2_dic[idx]:
        lst_subid.append(sub)  

100%|█████████████████████████████| 1058009/1058009 [00:07<00:00, 144384.88it/s]


In [17]:
pre2sub = pd.DataFrame(lst_subid)
pre2time = pd.DataFrame(lst_time)
pre2item = pd.DataFrame(lst_itemid)

pre2 = pd.concat([pre2sub, pre2time, pre2item], axis = 1)
pre2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pre2['TYPE'] = 'PRE'
pre2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,21,2134-09-11,3.380550e+08,PRE
1,21,2134-09-11,0.000000e+00,PRE
2,21,2134-09-11,6.074928e+06,PRE
3,21,2134-09-11,5.175710e+08,PRE
4,21,2134-09-12,5.175710e+08,PRE
...,...,...,...,...
4546390,99991,2185-01-04,4.560663e+08,PRE
4546391,99991,2185-01-05,4.560663e+08,PRE
4546392,99991,2185-01-04,5.817702e+10,PRE
4546393,99991,2185-01-05,5.817702e+10,PRE


## pro

- 변수 sub_list를 활용해 PROCEDUREEVENTS_MV.csv에서 데이터를 인덱싱하여 pre2 데이터프레임을 생성하는 과정입니다.

In [18]:
pro2 = pro.copy()
pro2 = pro2[(pro2['SUBJECT_ID'].isin(sub_list))]
pro2 = pro2[['SUBJECT_ID','STARTTIME','ENDTIME','ITEMID']]
pro2 = pro2.reset_index(drop=True)

date_only(pro2,'STARTTIME')
date_only(pro2,'ENDTIME')

# a = date_range(pro2['STARTTIME'][4], pro2['ENDTIME'][4]) #test
# b = date_range(pro2['STARTTIME'][4], pro2['ENDTIME'][4]) #test

In [19]:
pro2

,SUBJECT_ID,STARTTIME,ENDTIME,ITEMID
0,29070,2145-03-12,2145-03-12,225401
1,29070,2145-03-12,2145-03-12,225454
2,29070,2145-03-12,2145-03-18,225792
3,29070,2145-03-12,2145-03-12,225402
4,29070,2145-03-13,2145-03-16,224560
...,...,...,...,...
69178,58723,2192-09-05,2192-09-07,225202
69179,48872,2184-08-30,2184-09-02,224264
69180,48872,2184-08-30,2184-09-02,225752
69181,48872,2184-08-31,2184-09-02,224270


In [20]:
pro2_list = pro2.values.tolist()
pro2_dic = {}
lst_time = []
lst_itemid = []
lst_subid = []
for idx, row in enumerate(tqdm(pro2_list)):
    sub, start, end, itemid = row[0], row[1], row[2], row[3]
    
    day_list = date_range(start, end)
    
    # index에 맞춰서 subid, date, itemid를 넣어주기 위한 dic
    # {0: ['2149-11-09'], ...,1327249: ['2181-02-11', '2181-02-12']}
    pro2_dic[idx] = day_list
    
    # pro2 DATE열에 들어갈 lst
    lst_time.extend(pro2_dic[idx][:])

    # pro2 itemid(NDC)열에 들어갈 lst
    for _ in pro2_dic[idx]:
        lst_itemid.append(itemid)
    
    # pro2 subid열에 들어갈 lst
    for _ in pro2_dic[idx]:
        lst_subid.append(sub)

pro2sub = pd.DataFrame(lst_subid)
pro2time = pd.DataFrame(lst_time)
pro2item = pd.DataFrame(lst_itemid)

100%|█████████████████████████████████| 69183/69183 [00:00<00:00, 266928.28it/s]


In [21]:
pro2 = pd.concat([pro2sub, pro2time, pro2item], axis = 1)
pro2.columns = ['SUBJECT_ID', 'CHARTTIME', 'ITEMID']
pro2['TYPE'] = ["PRO"]*len(pro2)
pro2

,SUBJECT_ID,CHARTTIME,ITEMID,TYPE
0,29070,2145-03-12,225401,PRO
1,29070,2145-03-12,225454,PRO
2,29070,2145-03-12,225792,PRO
3,29070,2145-03-13,225792,PRO
4,29070,2145-03-14,225792,PRO
...,...,...,...,...
147551,48872,2184-09-02,225752,PRO
147552,48872,2184-08-31,224270,PRO
147553,48872,2184-09-01,224270,PRO
147554,48872,2184-09-02,224270,PRO


## merge

- 지금까지 생성한 lab2, pre2, pro2 데이터프레임을 병합하여 total_data 데이터프레임을 생성하는 과정입니다.

In [22]:
m1 = pd.merge(lab2, pre2, on=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'TYPE'], how='outer')
total_data = pd.merge(m1, pro2, on=['SUBJECT_ID', 'ITEMID', 'CHARTTIME', 'TYPE'], how='outer')
total_data = total_data.sort_values(['SUBJECT_ID','CHARTTIME']).reset_index(drop=True)
total_data = total_data.astype({'ITEMID':'int'})

In [23]:
total_data

,SUBJECT_ID,ITEMID,CHARTTIME,FLAG,TYPE
0,3,50912,2101-10-04,abnormal,LAB
1,3,50931,2101-10-04,abnormal,LAB
2,3,51006,2101-10-04,abnormal,LAB
3,3,51221,2101-10-04,abnormal,LAB
4,3,51222,2101-10-04,abnormal,LAB
...,...,...,...,...,...
7679185,99991,904150061,2185-01-05,NaN,PRE
7679186,99991,54839224,2185-01-05,NaN,PRE
7679187,99991,456066270,2185-01-05,NaN,PRE
7679188,99991,58177020211,2185-01-05,NaN,PRE


- 우선 total_data의 SUBJECT_ID와 ITEMID로 구성된 2차원 zero matrix를 생성하고, total_data의 값을 채워넣습니다. 
- 그 뒤, 생성된 matrix에서 같은 열의 모든 값을 더해 그 값이 0이 되는 열을 삭제합니다. 이는 분석에 영향을 주지 않을 데이터를 삭제해 데이터의 크기를 줄이기 위함입니다. 

In [24]:
# 제로 매트릭스 생성 

total_id = total_data['SUBJECT_ID'].unique()

itemid_uniq = list(total_data['ITEMID'].sort_values(ascending=True).unique())
itemid_uniq1 = list(map(str, itemid_uniq))
zero_matrix = np.zeros((len(total_id), len(itemid_uniq)))
df = pd.DataFrame(zero_matrix, columns=itemid_uniq1)
df['SUBJECT_ID'] = total_id
df = df.set_index('SUBJECT_ID')

In [25]:
df

,0,50803,50804,50805,50806,50808,50809,50811,50813,50814,...,72140045585,74300000068,74300000533,74312000660,74312001370,78112001103,79511050204,87701071218,87701083336,87701089415
SUBJECT_ID,,,,,,,,,,,,,,,,,,,,,
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# abnormal인 행만 인덱싱
ab = total_data[total_data['FLAG']=='abnormal']

# SUBJECT_ID와 CHARTTIME 두 열을 멀티 인덱스로 설정
multi_index = pd.MultiIndex.from_frame(ab[['SUBJECT_ID', 'CHARTTIME']].drop_duplicates())
df = pd.DataFrame(0, index=multi_index, columns=itemid_uniq1)

# itemid를 기반으로 제로 매트릭스에 1을 삽입
for idx, row in tqdm(ab.iterrows(), total = 2985239):
    df.at[(row['SUBJECT_ID'], row['CHARTTIME']), str(row['ITEMID'])] = 1

100%|██████████████████████████████| 2985239/2985239 [01:13<00:00, 40770.90it/s]


In [27]:
# df에서 열의합이 0인컬럼만 저장
cols_with_all_zeros = df.columns[df.sum(axis=0) == 0]
cols_with_all_zeros_int = cols_with_all_zeros.astype('int')

# cols_with_all_zeros의 값들 중 total_data의 'ITEMID' 열에 포함된 값들을 확인
included_items = total_data['ITEMID'][total_data['ITEMID'].isin(cols_with_all_zeros_int)].unique()

# total_data에서 해당 ITEMID를 가진 행을 제거
total_data_filtered = total_data[~total_data['ITEMID'].isin(included_items)]


- total_data에서 해당 ITEMID를 가진 행이 제거된 total_data_filtered로 다시 zero matrix를 만들어, 그 값을 채워넣습니다.
- 이 때 데이터프레임은 전과 달리 SUBJECT_ID와 CHARTTIME이 행, ITEMID가 열인 2차원 데이터프레임입니다.

In [28]:
# 제로 매트릭스 생성
itemid_uniq = list(total_data_filtered['ITEMID'].sort_values(ascending=True).unique())
itemid_uniq1 = list(map(str, itemid_uniq))
zero_matrix = np.zeros((len(total_data_filtered), len(itemid_uniq)))
df = pd.DataFrame(zero_matrix, columns=itemid_uniq1)
df.index = total_data_filtered['SUBJECT_ID'].values
df.index.name = 'SUBJECT_ID'

In [29]:
# abnormal인 행만 인덱싱
ab = total_data_filtered[total_data['FLAG']=='abnormal']

# 다시 SUBJECT_ID와 CHARTTIME 두 열을 멀티 인덱스로 설정
multi_index = pd.MultiIndex.from_frame(ab[['SUBJECT_ID', 'CHARTTIME']].drop_duplicates())
df = pd.DataFrame(0, index=multi_index, columns=itemid_uniq1)

# itemid를 기반으로 제로 매트릭스에 1을 삽입
for idx, row in tqdm(ab.iterrows(), total = 2985239):
    df.at[(row['SUBJECT_ID'], row['CHARTTIME']), str(row['ITEMID'])] = 1


/home/t8user1/.conda/envs/example/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
100%|██████████████████████████████| 2985239/2985239 [01:13<00:00, 40766.82it/s]


In [30]:
df.reset_index(inplace=True)

In [31]:
df

,SUBJECT_ID,CHARTTIME,50803,50804,50805,50806,50808,50809,50811,50813,...,51493,51494,51498,51507,51514,51515,51516,51517,51526,51529
0,3,2101-10-04,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,2101-10-05,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2101-10-06,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2101-10-07,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3,2101-10-11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176692,99991,2184-12-30,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176693,99991,2184-12-31,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176694,99991,2185-01-01,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
176695,99991,2185-01-03,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- 완성된 데이터프레임을 data.csv로 저장합니다.

In [32]:
# 사전학습 데이터 (전체 데이터)
df.to_csv('data.csv', index = False)